鸢尾花数据集，数据集包含3类共150调数据，每类含50个数据，每条记录含4个特征：花萼长度、花萼宽度、花瓣长度、花瓣宽度

过这4个 特征预测鸢尾花卉属于（iris-setosa, iris-versicolour, iris-virginica）中的哪一品种

In [1]:
!head -5 data/lris.csv

In [2]:
##创建环境

import findspark
findspark.init()
##############################################
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("KNNrdd") \
        .getOrCreate()

sc = spark.sparkContext

In [3]:
#获得距离
import math
def getDis(array1, array2):
    return math.sqrt(sum([pow(x-y, 2) for x, y in zip(array1, array2)]))
#    return math.sqrt(array1,zip(array2).map(lambda x:pow(x[0]-x[1])).sum)

In [4]:
K = 15
data = sc.textFile("data/lris.csv").map(lambda x:x.split(",")).filter(lambda x:x[1]!= "Sepal.Length" and len(x)==6).map(lambda x:(x[-1], [float(i) for i in x[:-1]]))


In [5]:
data.collect()

[('setosa', [1.0, 5.1, 3.5, 1.4, 0.2]),
 ('setosa', [2.0, 4.9, 3.0, 1.4, 0.2]),
 ('setosa', [3.0, 4.7, 3.2, 1.3, 0.2]),
 ('setosa', [4.0, 4.6, 3.1, 1.5, 0.2]),
 ('setosa', [5.0, 5.0, 3.6, 1.4, 0.2]),
 ('setosa', [6.0, 5.4, 3.9, 1.7, 0.4]),
 ('setosa', [7.0, 4.6, 3.4, 1.4, 0.3]),
 ('setosa', [8.0, 5.0, 3.4, 1.5, 0.2]),
 ('setosa', [9.0, 4.4, 2.9, 1.4, 0.2]),
 ('setosa', [10.0, 4.9, 3.1, 1.5, 0.1]),
 ('setosa', [11.0, 5.4, 3.7, 1.5, 0.2]),
 ('setosa', [12.0, 4.8, 3.4, 1.6, 0.2]),
 ('setosa', [13.0, 4.8, 3.0, 1.4, 0.1]),
 ('setosa', [14.0, 4.3, 3.0, 1.1, 0.1]),
 ('setosa', [15.0, 5.8, 4.0, 1.2, 0.2]),
 ('setosa', [16.0, 5.7, 4.4, 1.5, 0.4]),
 ('setosa', [17.0, 5.4, 3.9, 1.3, 0.4]),
 ('setosa', [18.0, 5.1, 3.5, 1.4, 0.3]),
 ('setosa', [19.0, 5.7, 3.8, 1.7, 0.3]),
 ('setosa', [20.0, 5.1, 3.8, 1.5, 0.3]),
 ('setosa', [21.0, 5.4, 3.4, 1.7, 0.2]),
 ('setosa', [22.0, 5.1, 3.7, 1.5, 0.4]),
 ('setosa', [23.0, 4.6, 3.6, 1.0, 0.2]),
 ('setosa', [24.0, 5.1, 3.3, 1.7, 0.5]),
 ('setosa', [25.0, 4.8, 3

In [6]:
#准备样本数据和测试数据
splits=data.randomSplit([0.7,0.3])
sampleData=splits[0]
testData=splits[1]

In [7]:
sampleDataBrc = sc.broadcast(sampleData.collect())

In [8]:
#获取距离最近的K个样本
def getPoint(elem,K):
    print("begin:elem")
#    dis = sampleDataBrc.value.map(lambda x:(getDis(elem[1],x[1]),x[0]))
    dis = [(getDis(elem[1], x[1]), x[0]) for x in sampleDataBrc.value]
#    dis = list(elem.map(lambda x:(getDis(elem[1],x[1]),x[0])))
    #获取距离最近的k个样本
    minDis = sorted(dis,key=lambda x:x[0],reverse=False)[:K]
    minDisLabel = [x[0] for x in minDis]
    print(minDis)
    #取出这k个样本的label并且获取出现最多的label即为测试数据的label
    labels = sorted(minDis,key=lambda x:minDisLabel.count(x[0]),reverse = True)
    return "{},{}=>{}".format(elem[0], elem[1], labels[0][1])

In [9]:
testData = testData.map(lambda x: getPoint(x, K))

In [10]:
testData.collect()

['setosa,[2.0, 4.9, 3.0, 1.4, 0.2]=>setosa',
 'setosa,[9.0, 4.4, 2.9, 1.4, 0.2]=>setosa',
 'setosa,[10.0, 4.9, 3.1, 1.5, 0.1]=>setosa',
 'setosa,[13.0, 4.8, 3.0, 1.4, 0.1]=>setosa',
 'setosa,[17.0, 5.4, 3.9, 1.3, 0.4]=>setosa',
 'setosa,[20.0, 5.1, 3.8, 1.5, 0.3]=>setosa',
 'setosa,[21.0, 5.4, 3.4, 1.7, 0.2]=>setosa',
 'setosa,[26.0, 5.0, 3.0, 1.6, 0.2]=>setosa',
 'setosa,[27.0, 5.0, 3.4, 1.6, 0.4]=>setosa',
 'setosa,[30.0, 4.7, 3.2, 1.6, 0.2]=>setosa',
 'setosa,[31.0, 4.8, 3.1, 1.6, 0.2]=>setosa',
 'setosa,[33.0, 5.2, 4.1, 1.5, 0.1]=>setosa',
 'setosa,[36.0, 5.0, 3.2, 1.2, 0.2]=>setosa',
 'setosa,[39.0, 4.4, 3.0, 1.3, 0.2]=>setosa',
 'setosa,[41.0, 5.0, 3.5, 1.3, 0.3]=>setosa',
 'setosa,[43.0, 4.4, 3.2, 1.3, 0.2]=>setosa',
 'versicolor,[51.0, 7.0, 3.2, 4.7, 1.4]=>versicolor',
 'versicolor,[61.0, 5.0, 2.0, 3.5, 1.0]=>versicolor',
 'versicolor,[62.0, 5.9, 3.0, 4.2, 1.5]=>versicolor',
 'versicolor,[74.0, 6.1, 2.8, 4.7, 1.2]=>versicolor',
 'versicolor,[75.0, 6.4, 2.9, 4.3, 1.3]=>versicolo